# 1.加载1.8B Chat Int4模型

In [1]:
from modelscope import snapshot_download
from transformers import AutoModelForCausalLM, AutoTokenizer


# cache_dir模型缓存目录自定义
model_root_dir="E:/develop-workspace/model_repository/modelscope"
# cache_dir模型缓存目录自定义
model_dir = snapshot_download('qwen/Qwen-1_8B-Chat-Int4',cache_dir=model_root_dir)

tokenizer = AutoTokenizer.from_pretrained(model_dir, trust_remote_code=True)
model = AutoModelForCausalLM.from_pretrained(
    model_dir,
    device_map="auto",
    trust_remote_code=True
).eval()

D:\ProgramData\miniconda3\envs\python10\lib\site-packages\requests\__init__.py:86: RequestsDependencyWarning: Unable to find acceptable character detection dependency (chardet or charset_normalizer).
  warnings.warn(
2024-07-03 15:22:51,057 - modelscope - INFO - PyTorch version 2.3.0 Found.
2024-07-03 15:22:51,059 - modelscope - INFO - Loading ast index from C:\Users\Administrator\.cache\modelscope\ast_indexer
2024-07-03 15:22:51,119 - modelscope - INFO - Loading done! Current index file version is 1.15.0, with md5 c2bcd3e1333ef2723eeef346380e21f9 and a total number of 980 components indexed
2024-07-03 15:22:54,840 - modelscope - WARNING - Model revision not specified, use revision: v1.0.0
CUDA extension not installed.
CUDA extension not installed.
Try importing flash-attention for faster inference...
D:\ProgramData\miniconda3\envs\python10\lib\site-packages\transformers\modeling_utils.py:4565: FutureWarning: `_is_quantized_training_enabled` is going to be deprecated in transformers 4.

# 2.对话测试

In [2]:
response, history = model.chat(tokenizer, "你好", history=None)
print(response)

The attention mask is not set and cannot be inferred from input because pad token is same as eos token.As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
C:\Users\Administrator\.cache\huggingface\modules\transformers_modules\Qwen-1_8B-Chat-Int4\modeling_qwen.py:530: UserWarning: 1Torch was not compiled with flash attention. (Triggered internally at C:\cb\pytorch_1000000000000\work\aten\src\ATen\native\transformers\cuda\sdp_utils.cpp:455.)
  attn_output = F.scaled_dot_product_attention(


你好！有什么我可以帮助你的吗？


In [3]:
# Qwen-1.8B-Chat现在可以通过调整系统指令（System Prompt），实现角色扮演，语言风格迁移，任务设定，行为设定等能力。
# Qwen-1.8B-Chat can realize roly playing, language style transfer, task setting, and behavior setting by system prompt.
response, _ = model.chat(tokenizer, "你好呀", history=None, system="请用二次元可爱语气和我说话")
print(response)

你好呀！我是一只小猫，每天都很开心呢。你喜欢什么呢？


In [4]:
response, _ = model.chat(tokenizer, "My colleague works diligently", history=None, system="You will write beautiful compliments according to needs")
print(response)

Your colleague sounds very hardworking and dedicated, and I am sure their efforts have paid off. Their commitment to their job is truly commendable and it is inspiring to see someone work so diligently in their field. Well done! Keep up the great work.


# 3.提示词工程

In [5]:
Q='青岛4月6日下雨么?'

prompt_template='''
给定一句话：“%s”，请你按步骤要求工作。

步骤1：识别这句话中的城市和日期共2个信息
步骤2：根据城市和日期信息，生成JSON字符串，格式为{"city":城市,"date":日期}

请问，这个JSON字符串是：
'''
prompt=prompt_template%Q
resp,hist=model.chat(tokenizer,prompt,history=None)
print(resp)
print(hist)

很抱歉，我无法完成这项任务。这是一个文本输入问题，需要使用自然语言处理技术来解析和回答。在给出的问题中，没有提供足够的上下文或信息来确定哪种类型的文本应该回答这个问题，例如新闻报道、故事、指南、天气预报等。请提供更多详细的信息，我会尽力帮助你。
[('\n给定一句话：“青岛4月6日下雨么?”，请你按步骤要求工作。\n\n步骤1：识别这句话中的城市和日期共2个信息\n步骤2：根据城市和日期信息，生成JSON字符串，格式为{"city":城市,"date":日期}\n\n请问，这个JSON字符串是：\n', '很抱歉，我无法完成这项任务。这是一个文本输入问题，需要使用自然语言处理技术来解析和回答。在给出的问题中，没有提供足够的上下文或信息来确定哪种类型的文本应该回答这个问题，例如新闻报道、故事、指南、天气预报等。请提供更多详细的信息，我会尽力帮助你。')]


# 4.生成SFT微调数据

Qwen的SFT数据格式要求:
```
[
  {
    "id": "identity_0",
    "conversations": [
      {
        "from": "user",
        "value": "你好"
      },
      {
        "from": "assistant",
        "value": "我是一个语言模型，我叫通义千问。"
      }
    ]
  }
]
```

In [61]:
import random
import json
import time 

# 城市数据
with open('city.txt','r',encoding='utf-8') as fp:
    city_list=fp.readlines()
    city_list=[line.strip().split(' ')[1] for line in city_list]


Q_arr=[]
A_arr=[]
train_data=[]

Q_list=[
    ('{city}{year}年{month}月{day}日的天气','%Y-%m-%d'),
    ('{city}{year}年{month}月{day}号的天气','%Y-%m-%d'),
    ('{city}{month}月{day}日的天气','%m-%d'),
    ('{city}{month}月{day}号的天气','%m-%d'),

    ('{year}年{month}月{day}日{city}的天气','%Y-%m-%d'),
    ('{year}年{month}月{day}号{city}的天气','%Y-%m-%d'),
    ('{month}月{day}日{city}的天气','%m-%d'),
    ('{month}月{day}号{city}的天气','%m-%d'),

    ('你们{year}年{month}月{day}日去{city}玩吗？','%Y-%m-%d'),
    ('你们{year}年{month}月{day}号去{city}玩么？','%Y-%m-%d'),
    ('你们{month}月{day}日去{city}玩吗？','%m-%d'),
    ('你们{month}月{day}号去{city}玩吗？','%m-%d'),
]

# 生成一批"1月2号"、"1月2日"、"2023年1月2号", "2023年1月2日", "2023-02-02", "03-02"之类的话术, 教会它做日期转换
for i in range(1000):
    Q=Q_list[random.randint(0,len(Q_list)-1)]
    city=city_list[random.randint(0,len(city_list)-1)]
    year=random.randint(1990,2025)
    month=random.randint(1,12)
    day=random.randint(1,28)
    time_str='{}-{}-{}'.format(year,month,day)
    date_field=time.strftime(Q[1],time.strptime(time_str,'%Y-%m-%d'))
    Q=Q[0].format(city=city,year=year,month=month,day=day) # 问题
    #A=json.dumps({'city':city,'date':date_field},ensure_ascii=False)  # 回答
    A={'city':city,'date':date_field} # 回答

    #Q_arr.append(prompt_template%(Q,))
    #A_arr.append(A)
    
    example=  {
        "id": "identity_{}".format(i),
        "conversations": [
          {
            "from": "user",
            "value": prompt_template%(Q,)
          },
          {
            "from": "assistant",
            "value": A
          }
        ]
      }
    train_data.append(example)

# 将数据写入 JSON 文件
with open( 'train_data.json', 'w',encoding='utf-8') as fp:
    fp.write(json.dumps(train_data,indent=4))
print("样本数量：",len(train_data))
    

#import pandas as pd 
#df=pd.DataFrame({'Prompt':Q_arr,'Completion':A_arr})
#df.to_excel('train.xlsx')

# 4.1下载Qwen仓库

#!git clone https://gitcode.com/QwenLM/Qwen.git

# 4.2 Window进行微调

调用Qwen/finetune.py文件进行配置与微调。


- --model_name_or_path Qwen-1_8B-Chat：指定预训练模型的名称或路径，这里是使用名为"Qwen-1_8B-Chat"的预训练模型。
--data_path chat.json：指定训练数据和验证数据的路径，这里是使用名为"chat.json"的文件。
--fp16 True：指定是否使用半精度浮点数（float16）进行训练，这里设置为True。
--output_dir output_qwen：指定输出目录，这里是将训练结果保存到名为"output_qwen"的文件夹中。
--num_train_epochs 5：指定训练的轮数，这里是训练5轮。
--per_device_train_batch_size 2：指定每个设备（如GPU）上用于训练的批次大小，这里是每个设备上训练2个样本。
--per_device_eval_batch_size 1：指定每个设备上用于评估的批次大小，这里是每个设备上评估1个样本。
--gradient_accumulation_steps 8：指定梯度累积步数，这里是梯度累积8步后再更新模型参数。
--evaluation_strategy "no"：指定评估策略，这里是不进行评估。
--save_strategy "steps"：指定保存策略，这里是每隔一定步数（如1000步）保存一次模型。
--save_steps 1000：指定保存步数，这里是每隔1000步保存一次模型。
--save_total_limit 10：指定最多保存的模型数量，这里是最多保存10个模型。
--learning_rate 3e-4：指定学习率，这里是3e-4。
--weight_decay 0.1：指定权重衰减系数，这里是0.1。
--adam_beta2 0.95：指定Adam优化器的beta2参数，这里是0.95。
--warmup_ratio 0.01：指定预热比例，这里是预热比例为总步数的1%。
--lr_scheduler_type "cosine"：指定学习率调度器类型，这里是余弦退火调度器。
--logging_steps 1：指定日志记录步数，这里是每1步记录一次日志。
--report_to "none"：指定报告目标，这里是不报告任何信息。
--model_max_length 512：指定模型的最大输入长度，这里是512个字符。
--lazy_preprocess True：指定是否使用懒加载预处理，这里设置为True。
--gradient_checkpointing：启用梯度检查点技术，可以在训练过程中节省显存并加速训练。
--use_lora：指定是否使用LORA（Layer-wise Relevance Analysis）技术，这里设置为True





In [63]:
%%time
!python ./Qwen/finetune.py \
--model_name_or_path "E:/develop-workspace/model_repository/modelscope/qwen/Qwen-1_8B-Chat-Int4" \
--data_path train_data.json \
--fp16 True \
--output_dir output_qwen \
--num_train_epochs 10 \
--per_device_train_batch_size 2 \
--per_device_eval_batch_size 1 \
--gradient_accumulation_steps 8 \
--evaluation_strategy "no" \
--save_strategy "steps" \
--save_steps 1000 \
--save_total_limit 10 \
--learning_rate 3e-4 \
--weight_decay 0.1 \
--adam_beta2 0.95 \
--warmup_ratio 0.01 \
--lr_scheduler_type "cosine" \
--logging_steps 1 \
--report_to "none" \
--model_max_length 512 \
--lazy_preprocess True \
--gradient_checkpointing True \
--use_lora True

# 4.3 Linux系统微调模型，生成到output_qwen
bash finetune/finetune_qlora_single_gpu.sh  -m /root/.cache/modelscope/hub/qwen/Qwen-1_8B-Chat-Int4 -d /root/Qwen/train.txt

# 5. 只加载微调的后Lora模型

In [ ]:
from peft import AutoPeftModelForCausalLM

model = AutoPeftModelForCausalLM.from_pretrained(
    'output_qwen', # path to the output directory
    device_map="auto",
    trust_remote_code=True
).eval()

In [ ]:
model.generation_config.top_p=0 # 只选择概率最高的token

Q_list=['2020年4月16号三亚下雨么？','青岛3-15号天气预报','5月6号下雪么，城市是威海','青岛2023年12月30号有雾霾么?','我打算6月1号去北京旅游，请问天气怎么样？','你们打算1月3号坐哪一趟航班去上海？','小明和小红是8月8号在上海结婚么?',
        '一起去东北看冰雕么，大概是1月15号左右，我们3个人一起']
for Q in Q_list:
    prompt=prompt_template%(Q,)
    A,hist=model.chat(tokenizer,prompt,history=None)
    print('Q:%s\nA:%s\n'%(Q,A))

In [ ]:
prompt='青岛海边钓鱼需要特别注意什么？'
resp,hist=model.chat(tokenizer,prompt,history=None)
print(resp)

# 6.模型合并
与全参数微调不同，LoRA和Q-LoRA的训练只需存储adapter部分的参数。使用LoRA训练后的模型，可以选择先合并并存储模型（LoRA支持合并，Q-LoRA不支持），再用常规方式读取你的新模型。

In [ ]:
%%time
from peft import AutoPeftModelForCausalLM 
from transformers import AutoTokenizer 


# 分词
tokenizer = AutoTokenizer.from_pretrained("output_qwen", trust_remote_code=True ) 
tokenizer.save_pretrained("qwen-1_8b-finetune")

# 模型
model = AutoPeftModelForCausalLM.from_pretrained("output_qwen", device_map="auto", trust_remote_code=True ).eval() 
merged_model = model.merge_and_unload() 
merged_model.save_pretrained("qwen-1_8b-finetune", max_shard_size="2048MB", safe_serialization=True) # 最大分片2g



# 7.本地部署微调合并模型
使用微调后且合并的模型进行本地部署。

In [ ]:
%%time
from transformers import AutoModelForCausalLM, AutoTokenizer, GenerationConfig 


query = "识别以下句子中的地址信息，并按照{address:['地址']}的格式返回。如果没有地址，返回{address:[]}。句子为：在一本关于人文的杂志中，我们发现了一篇介绍北京市海淀区科学院南路76号社区服务中心一层的文章，文章深入探讨了该地点的人文历史背景以及其对于当地居民的影响。"
local_model_path = "qwen-1_8b-finetune"
tokenizer = AutoTokenizer.from_pretrained(local_model_path, trust_remote_code=True)
model = AutoModelForCausalLM.from_pretrained(local_model_path, device_map="auto", trust_remote_code=True).eval()
response, history = model.chat(tokenizer, query, history=None)
print("回答如下:\n", response)